<img src="https://raw.githubusercontent.com/seismo-live/seismo_live/refs/heads/master/notebooks/Workshops/2017_Baku_STCU_IRIS_ObsPy_course/images/obspy_logo_full_524x179px.png" width=90%> 

In this notebooks we will look at the Anak Krakatau eruption from 2018 in more detail. We will use the data that you downloaded in the FDSN exercise.

In [14]:
try:
  import obspy
  import cartopy
except ModuleNotFoundError:
  !pip -qq install obspy cartopy
  import cartopy
  import obspy

In [ ]:
!wget https://github.com/yannikbehr/BMKG_ObsPy_workshop/raw/refs/heads/main/data/indonesia_waveforms.tar.gz
!tar -xzf indonesia_waveforms.tar.gz

Let's start again by first loading the catalogue around the eruption time at 2018-12-22T13:55:00. We will use Geofon's catalogue as IRIS' catalogue doesn't contain this event.

In [ ]:
from obspy import read, read_events, UTCDateTime
from obspy.clients.fdsn import Client

t = UTCDateTime("2018-12-22T13:55:00")
client = Client("Geofon")
catalog = client.get_events(starttime=t-180, endtime=t + 1800,
                            minmagnitude=2)
print(catalog)
catalog.plot();

We are interested in the M5.1 event. Select the event from the catalog and print its latitude, longitude and depth using the `preferred_origin()` method.

In [ ]:
event = catalog[-1]
event_lat = event.preferred_origin().latitude
event_lon = event.preferred_origin().longitude
event_depth = event.preferred_origin().depth
print(event_lat, event_lon, event_depth)

Now read in the waveform data `data/anak_krakatau_2018.mseed` into a `stream`. 

In [ ]:
st = read('data/anak_krakatau_2018.mseed')
print(st)

Next we have to download the instrument response for each station which we will need for the magnitude computation later on. To do so use the `level=response` keyword argument to [get_stations](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.client.Client.get_stations.html#obspy.clients.fdsn.client.Client.get_stations).

In [ ]:
from obspy import Inventory
inv = Inventory()
for tr in st:
    inv_ = client.get_stations(network=tr.stats.network, station=tr.stats.station,
                               channel=tr.stats.channel, location=tr.stats.location,
                               starttime=tr.stats.starttime, endtime=tr.stats.endtime,
                               level="response")
    inv += inv_
print(inv)

Now we want to plot a record section of the data as shown [here](https://docs.obspy.org/tutorial/code_snippets/waveform_plotting_tutorial.html#plotting-a-record-section). To do so we first need to compute the distance of each station to the event in meters. You can use the function [gps2dist_azimuth](https://docs.obspy.org/packages/autogen/obspy.geodetics.base.gps2dist_azimuth.html#obspy.geodetics.base.gps2dist_azimuth) from the [obspy.geodetics](https://docs.obspy.org/packages/obspy.geodetics.html#module-obspy.geodetics) module.

In [20]:
from obspy.geodetics import gps2dist_azimuth

for tr in st:
    station_lat = inv.select(station=tr.stats.station).networks[0].stations[0].latitude
    station_lon = inv.select(station=tr.stats.station).networks[0].stations[0].longitude
    dist, az, baz = gps2dist_azimuth(event_lat, event_lon,
                                     station_lat, station_lon)
    tr.stats.distance = dist

Next let's calculate some rough theoretical travel times for the P-wave using the ak135 earth model. You can do that using the [obspy.taup](https://docs.obspy.org/packages/obspy.taup.html#module-obspy.taup) module.

In [21]:
from obspy.taup import TauPyModel
model = TauPyModel(model="ak135")

In [ ]:
from obspy.geodetics import kilometer2degrees
print(event.preferred_origin().time)
for tr in st:
    distance = kilometer2degrees(tr.stats.distance/1000)
    arrivals = model.get_ray_paths(source_depth_in_km=0,
                                    distance_in_degree=distance,
                                    phase_list=["P"])
    tr.stats.p_arrival = event.preferred_origin().time + arrivals[0].time
    print(tr.id, event.preferred_origin().time + arrivals[0].time)

You may have noticed that you get five different travel times for the P-wave. To find out why plot the ray paths using the [get_ray_paths](https://docs.obspy.org/packages/obspy.taup.html#module-obspy.taup) and [plot_rays](https://docs.obspy.org/packages/obspy.taup.html#module-obspy.taup) methods.

In [ ]:
ray_paths = model.get_ray_paths(0, distance, phase_list=["P"])
ray_paths.plot_rays(plot_type="cartesian")

Now you have everything to plot a record section for the data. For an extra challenge try to add station names and theoretical first arrival times. You can get some ideas how to do that by looking at the source code of the [example](https://docs.obspy.org/tutorial/code_snippets/waveform_plotting_tutorial.html#plotting-a-record-section).

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.transforms import blended_transform_factory

fig = plt.figure()
st.detrend("demean")
st.plot(type='section', time_down=True, fig=fig, norm_method='stream')
ax = fig.axes[0]
transform = blended_transform_factory(ax.transData, ax.transAxes)
for tr in st:
    ax.text(tr.stats.distance / 1e3, 1.0, tr.stats.station, rotation=270,
            va="bottom", ha="center", transform=transform, zorder=10)
    ax.plot([tr.stats.distance/1e3 - 10, tr.stats.distance/1e3 + 10],
            [tr.stats.p_arrival - tr.stats.starttime, tr.stats.p_arrival - tr.stats.starttime],
            color='red', lw=0.5)

To calculate the magnitude first plot the [spectrograms](https://docs.obspy.org/packages/obspy.imaging.html#module-obspy.imaging) so you can get an idea of the frequency content of the surface waves.

In [ ]:
st.spectrogram(wlen=100);

Now you have all the information to compute the magnitude of the event. We will use [estimate_magnitude](https://docs.obspy.org/packages/autogen/obspy.signal.invsim.estimate_magnitude.html#obspy.signal.invsim.estimate_magnitude) to do so. Can you think of reasons why the magnitude was underestimated?

In [ ]:
from obspy.signal.invsim import estimate_magnitude

for station in ['BBJI', 'PMBI']:
    response = inv.get_response(seed_id=f"GE.{station}..LHZ", datetime=event.preferred_origin().time)
    tr = st.select(station=station)[0]
    max_amplitude = tr.data.max()
    magnitude = estimate_magnitude(response, 2*abs(max_amplitude), 1.5, tr.stats.distance/1e3)
    print(magnitude)
